# Image enhancement

In the example, the use of techniques for constrast enhancement is explored.

- The method of histogram equalization automatically increases the contrast between the structures in the image. The point-to-point transformation function of the image is given by the cumulative distribution function of the original image, which is estimated as the cuulative histogram. As a result, the histogram of the obtained image shows a greater distribution of the pixels in the intensity range between 0 and 1, eliminating the original concentration of the pixels in a narrower range of intensities.

- Another technique that pursues contrast enhancement is to expand the histogram of the original image by applying a transformation function. The simplest function to obtain this result is the linear transformation, which maps pixels in the range [a, b] to the interval [0, 1]. Every pixel below a is assigned 0, and every pixel with an intensity greater than b is designated a value of 1. For instance, the percentiles 2 and 98 can be used for the definition of the interval [a, b].

# 1. Importing libraries to use during the exercise

- numpy. Working with arrays.
- matplotlib. Plotting signals.
- skimage. Image processing.

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float, io
from skimage import exposure

# 2. Function to plot an image, its histogram and its cumulative histogram

The purpose of this function is to show the three elements of interest in this exercise:

- An original image. It will be the image we want to enchance.
- Histogram. The histogram of the original image will allow us to identify if its pixels are specially concentrated in a narrow interval of intensities.
- Cumularive histogram. In addition to the histogram of the image, the cumulative histogram, which is used as an estimation of the cumulative distribution function of the image, is plotted.

In [2]:
def plot_img_and_hist(img, axes, bins=256):
    
    # img: image to be plotted together with its histogram and cumulative histogram.
    # axes: position of the plots
    
    # Plot an image along with its histogram and cumulative histogram
    
    # Pixel values casted as float
    img = img_as_float(img)
    
    # Axes for the image (ax_img) and its histogram and cumulative histogram (ax_hist, ax_cdf)
    ax_img, ax_hist = axes
    ax_cdf = ax_hist.twinx()

    # Display the original image
    ax_img.imshow(img, cmap=plt.cm.gray)
    ax_img.set_axis_off()
    ax_img.set_adjustable('box-forced')

    # Display histogram
    ax_hist.hist(img.ravel(), bins=bins, histtype='step', color='black')
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
    ax_hist.set_xlabel('Pixel intensity')
    ax_hist.set_xlim(0, 1)
    ax_hist.set_yticks([])

    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(img, bins)
    ax_cdf.plot(bins, img_cdf, 'r')
    ax_cdf.set_yticks([])

    return ax_img, ax_hist, ax_cdf




# 3. Image enhancement using contrast stretching

In this case, a pixel-wise transformation function is used to span the histogram of the original image. In other words, the intensity of low-intensity pixels is reduced and the intensity of high-intensity pixels is increased.

In [3]:
# Contrast stretching

lena = io.imread(fname="/Users/victor/lowcontrastlena.gif", as_grey=True)
# lena = data.moon()

# Intensity values higher than uplim will be set to the max intensity (1 or 255)
# Intensity values smaller than uplim will be set to the min intensity (0)

lowlim, uplim = np.percentile(lena, (0.05, 98.5))
lena_rescale = exposure.rescale_intensity(lena, in_range=(lowlim, uplim))

lenamin=np.min(lena)
lenamax=np.max(lena)

print "Min of the original image: ", lenamin, " - Max of the original image: ", lenamax, " - Lower intensity limit: ", lowlim, " - Upper intensity limit: ", uplim

inputin = np.linspace(start=lowlim, stop=uplim, num=500)
outputin = np.linspace(start=0, stop=255, num = 500)

f, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 12))
ax0.imshow(lena, cmap='gray', aspect='equal')
ax1.imshow(lena_rescale, cmap='gray', aspect='equal')
ax2.plot(inputin, outputin)
ax2.set_ylim([0, 255])
ax2.set_xlim([0, 255])
ax2.set_xlabel('Input')
ax2.set_ylabel('Output')
ax2.set_title('Transform function')
ax2.figure.set_figheight(5)
plt.tight_layout()
plt.gca().set_aspect('equal', adjustable='datalim')
plt.show()



SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Min of the original image: ", lenamin, " - Max of the original image: ", lenamax, " - Lower intensity limit: ", lowlim, " - Upper intensity limit: ", uplim)? (<ipython-input-3-9600316a7a7f>, line 15)

In [ ]:
# Due to contrast stretching some high frequency noise is observed in the image
# A Gaussian low-pass filter is applied to the image to remove these artifacts

from skimage.morphology import disk
from skimage.filters import median, gaussian

lena_rescale_filt = gaussian(lena_rescale, sigma=0.75)

io.imshow(lena_rescale_filt, cmap='gray', aspect='equal')
io.show()

# 4. Comparing contrast strtching and histogram equalization

Once contrast stretching has been tried, we propose the comparison of the obtained result with that derived from another common technique for image enhancement such as histogram equalization.

In [ ]:
# Load an example image
#img = data.moon()
img=lena

# Contrast stretching: expanding the original histogram by using a linear transform function
lowlim, uplim = np.percentile(img, (2, 98))
img_rescale = exposure.rescale_intensity(img, in_range=(lowlim, uplim))

# Equalization: expanding the original histogram by using the cdf of the original image
img_eq = exposure.equalize_hist(img)

# Displaying the results
fig = plt.figure(figsize=(8, 5))
axes = np.zeros((2, 3), dtype=np.object)
axes[0, 0] = fig.add_subplot(2, 3, 1)
for i in range(1, 3):
    axes[0, i] = fig.add_subplot(2, 3, 1+i, sharex=axes[0,0], sharey=axes[0,0])
for i in range(0, 3):
    axes[1, i] = fig.add_subplot(2, 3, 4+i)

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img, axes[:, 0])
ax_img.set_title('Low contrast image')

y_min, y_max = ax_hist.get_ylim()
ax_hist.set_ylabel('Number of pixels')
ax_hist.set_yticks(np.linspace(0, y_max, 5))

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_rescale, axes[:, 1])
ax_img.set_title('Contrast stretching')

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_eq, axes[:, 2])
ax_img.set_title('Histogram equalization')

ax_cdf.set_ylabel('Fraction of total intensity')
ax_cdf.set_yticks(np.linspace(0, 1, 5))

# prevent overlap of y-axis labels
fig.tight_layout()
plt.show()

# Using GIMP for contrast enhancement

This kind of transformations are included in most of the software tools for image processing. Here, we propose the use of GIMP, which is an open source tool for advanced image processing and edition.

- Definition of point-to-point transformation functions: Colores -> Curvas.

- Contrast enhancement by using a linear transformation: Colores -> Niveles: Definir Niveles de Entrada [a, b] y Niveles de Salida.

- Histogram equalization: Colores -> Auto -> Ecualizar
